In [ ]:
# %pip install biopython filelock

In [ ]:
# %pip install idtrack

In [ ]:
import scanpy as sc
import pandas as pd
import numpy as np
import warnings
import anndata as ad
warnings.filterwarnings("ignore")
from scipy.sparse import csr_matrix
from IPython.display import display, HTML

In [ ]:
adata_sc = sc.read_h5ad('../single_cell_int/adata_sc_int_outlier.h5ad')
# adata_sn = sc.read_h5ad('../single_nuc_int/adata_nuc_int_outlier.h5ad')

In [ ]:
import pickle
with open('kemal_map_result.pickle', 'rb') as f:
    results_dict = pickle.load(f)

pd.DataFrame.from_dict(results_dict['binned_conversions_adata_sc_genes'], orient='index').T

df = pd.DataFrame(results_dict['matching_adata_sc_genes']) #.groupby('final_database').size()

df_subset = df[['last_node', 'query_id']]

# Assuming df_subset['last_node'] contains a list of tuples
df_subset['col1'] = df_subset['last_node'].apply(lambda x: x[0][0] if len(x) > 0 else None)
df_subset['col2'] = df_subset['last_node'].apply(lambda x: x[0][1] if len(x) > 0 else None)

df_subset

from collections import defaultdict
ensembl_dict = defaultdict(set)
for index, row in df_subset.iterrows():
    ensembl_dict[row['col1']].add(row['query_id'])
    # ensembl_dict[row['col2']].add(row['query_id'])
filtered_dict = {key: list(value) for key, value in ensembl_dict.items() if len(value) > 1}

filtered_dict.pop(None)

mapped_df = pd.DataFrame.from_dict(filtered_dict, orient='index')

dict_map = dict(list(zip(df_subset.query_id, df_subset.col1)))

adata_sc.var['ensembl_id'] = adata_sc.var_names.map(dict_map)

len(adata_sc.var.ensembl_id.unique())

var_names = pd.Series(adata_sc.var.ensembl_id)
unique_ensembl_ids = var_names.unique()

# new_X = csr_matrix((adata_sc.X.shape[0], len(unique_ensembl_ids)))

# new_X

In [ ]:
# for i, ensembl_id in enumerate(unique_ensembl_ids):
#     indices = np.where(var_names == ensembl_id)[0]    
#     new_X[:, i] = adata_sc.X[:, indices].sum(axis=1)
#TODO remove if not needed

# look at the mapped df to see how many genes are mapped to one ensembl ID

In [ ]:
mapped_df['None_count'] = mapped_df.isna().sum(axis=1)


In [ ]:
mapped_df.sort_values('None_count')

In [ ]:
unique_genes = len(set(mapped_df.iloc[:, :14].values.flatten()))
ensembl_ids = mapped_df.sort_values('None_count').shape[0]

# Format and display the text with larger font
display(HTML(f'<h3>{unique_genes} unique genes are mapped to {ensembl_ids} ensembl IDs</h3>'))

# Change adata_sc.var

In [ ]:
adata_sc.var.ensembl_id = adata_sc.var.ensembl_id.astype(str)
# sc.get.aggregate(adata_sc, by='ensembl_id', func='sum', axis=1)
adata_sc.var.reset_index(inplace=True)
adata_sc.var_names = adata_sc.var.ensembl_id
adata_sc.var.rename(columns={'index':'gene_name'}, inplace=True)

# lots are mappeed to None, check for the reason why

In [ ]:
no_hits = adata_sc.var[adata_sc.var_names == 'None'].sort_values(by='total_counts', ascending=False)

In [ ]:
no_hits

In [ ]:
# take for example AC006273.7

In [ ]:
adata_sc.var[adata_sc.var.gene_name.str.contains('AC006273.7')]

In [ ]:
#check in original df
df[df.query_id == 'AC006273.7']

# google search for this gene
# turns out .7 is added for some reason, no hits with exact gene name on NCBI
# hit with AC006273
https://www.ncbi.nlm.nih.gov/gene/5725

# most get hits when '.number' is removed

In [ ]:
df[df.query_id.str.contains('AC006273')]

In [ ]:
df[df.query_id.str.contains('MTRNR2')]

In [ ]:
# try the genes with no ensemble IDs to map with mygene

In [ ]:
import mygene

In [ ]:
mg = mygene.MyGeneInfo()

In [ ]:
gene_name = no_hits['gene_name']

In [ ]:
mg.query(gene_name, scopes='symbol', fields='ensembl.gene', species='human')

In [ ]:
def get_ensembl_id(gene_name):
    try:
        result = mg.query(gene_name, scopes='symbol', fields='ensembl.gene', species='human')
        if 'hits' in result and len(result['hits']) > 0:
            return result['hits'][0]['ensembl']['gene'] if 'ensembl' in result['hits'][0] else None
    except:
        return None

# Apply the function to the 'gene_name' column to get Ensembl IDs
no_hits['ensembl_id'] = no_hits['gene_name'].apply(get_ensembl_id)

In [ ]:
no_hits

In [ ]:
df_subset[df_subset.col1.astype(str) == 'None']

In [ ]:
import re

# Function to remove suffixes from gene names
def remove_suffix(gene_name):
    return re.sub(r'\.\d+$', '', gene_name)

# Apply the function to the 'query_id' column
no_hits['query_id_clean'] = no_hits['gene_name'].apply(remove_suffix)

In [ ]:
no_hits.head()

In [ ]:
# Re-query with cleaned gene names
gene_names_clean = no_hits['query_id_clean'].dropna().unique().tolist()
results_clean = mg.querymany(gene_names_clean, scopes='symbol', fields='ensembl.gene', species='human')

# Convert results to a dictionary for quick lookup
mapping_clean = {item['query']: item.get('ensembl', {}).get('gene') for item in results_clean if 'ensembl' in item}

# Map the results back to the DataFrame
no_hits['ensembl_id_clean'] = no_hits['query_id_clean'].map(mapping_clean)

In [ ]:
no_hits.ensembl_id_clean.unique()

In [ ]:
results_alt = mg.querymany(no_hits['query_id_clean'], scopes=['alias', 'accession'], fields='ensembl.gene', species='human')

In [ ]:
import pandas as pd
import mygene

# Initialize the mygene query service
mg = mygene.MyGeneInfo()

# Clean up query IDs by removing suffixes
def remove_suffix(gene_name):
    return re.sub(r'\.\d+$', '', gene_name)

# no_hits['query_id_clean'] = no_hits['query_id'].apply(remove_suffix)

# Convert cleaned gene names to a list
gene_names_clean = no_hits['query_id_clean'].dropna().unique().tolist()

# Query HGNC using the cleaned gene names
results = mg.querymany(gene_names_clean, scopes='hgnc.symbol', fields='ensembl.gene', species='human')

# Convert results to a dictionary for quick lookup
mapping = {item['query']: item.get('ensembl', {}).get('gene') for item in results if 'ensembl' in item}

# Map the results back to the DataFrame
no_hits['ensembl_id'] = no_hits['query_id_clean'].map(mapping)


In [ ]:
no_hits.head()

In [ ]:
pwd

In [ ]:
# load ensemble csv
ensemble_csv = pd.read_table('../result.txt', header=None, names=column_names)

In [ ]:
attributes = """
<Attribute name = "ensembl_gene_id" />
<Attribute name = "ensembl_transcript_id" />
<Attribute name = "external_synonym" />
<Attribute name = "external_transcript_name" />
<Attribute name = "external_gene_name" />
<Attribute name = "entrezgene_id" />
<Attribute name = "hgnc_id" />
<Attribute name = "end_position" />
<Attribute name = "start_position" />
<Attribute name = "chromosome_name" />
<Attribute name = "ensembl_gene_id_version" />
<Attribute name = "ensembl_transcript_id_version" />
"""

column_names = re.findall(r'Attribute name = "(.*?)"', attributes)

In [ ]:
ensemble_csv.head()

In [ ]:
' '.join(no_hits['gene_name'].tolist())

In [ ]:
# no way ensemble mapping is possible

In [ ]:
len(set(adata_sc.var_names))

In [ ]:
len(set(adata_sc.var_names) & set(ensemble_csv['ensembl_gene_id_version']))

In [ ]:
len(set(adata_sc.var_names)) - len(set(adata_sc.var_names) & set(ensemble_csv['ensembl_gene_id_version'])) 

In [ ]:
# 607 genes missing ensemble IDs
# add the start and end position to adata

In [ ]:
dict_start = dict(list(zip(ensemble_csv['ensembl_gene_id_version'], ensemble_csv['start_position'])))
dict_end = dict(list(zip(ensemble_csv['ensembl_gene_id_version'], ensemble_csv['end_position'])))
dict_chr = dict(list(zip(ensemble_csv['ensembl_gene_id_version'], ensemble_csv['chromosome_name'])))

In [ ]:
adata_sc.var['start'] = adata_sc.var_names.map(dict_start)
adata_sc.var['end'] = adata_sc.var_names.map(dict_end)
adata_sc.var['chromosome'] = adata_sc.var_names.map(dict_chr)

In [ ]:
adata_sc.var['chromosome'] = 'chr' + adata_sc.var['chromosome']

In [ ]:
adata_sc.var_names = adata_sc.var.gene_name

In [ ]:
adata_sc.var.ensembl_id.unique().tolist()

In [ ]:
adata_sc.write('../single_cell_int/adata_nuc_int_outlier_genes.h5ad')

# Transfer ENSEMBLE IDs to snRNA_seq

In [ ]:
adata_sc = sc.read_h5ad('../single_cell_int/adata_sc_int_outlier_genes.h5ad')
adata_sn = sc.read_h5ad('../single_nuc_int/adata_nuc_int_outlier.h5ad')

In [ ]:
adata_sc.obs.Dataset.unique().tolist()

In [ ]:
sc_var = adata_sc.var[['ensembl_id', 'start', 'end', 'chromosome']]
sc_var['var_names'] = adata_sc.var.index

In [ ]:
sn_var = adata_sn.var.copy()
sn_var['var_names'] = adata_sn.var.index

In [ ]:
sn_var.head()

In [ ]:
sc_var.head()

In [ ]:
merged_var = pd.merge(sn_var, sc_var, on='var_names', how='left')

In [ ]:
adata_sn.var.head()

In [ ]:
merged_var

In [ ]:
adata_sn.var = merged_var.set_index('var_names')

In [ ]:
adata_sn.obs.rename(columns={'Dataset_Barcode': 'Dataset_Barcode_repeated'}, inplace=True)

In [ ]:
adata_sn.write('../single_nuc_int/adata_nuc_int_outlier_genes.h5ad')